In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models

In [5]:
ds_url = 'https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv'

In [91]:
df = pd.read_csv('/Users/lippd/Desktop/ML in R/Python/Text classification/data/cars.csv', sep=";")
df

,marka,marka_pop
0,ACURA,ACURA
1,ACURA,ACURA
2,ACURA,ACURA
3,ACURA,ACURA
4,ACURA,ACURA
...,...,...
12534,WAZ,LADA
12535,WAZ,LADA
12536,WAZ,LADA
12537,WAZ,LADA


In [128]:
train_size = int(len(df) * .8)

In [129]:
def train_test_split(df, train_size):
    train = df[:train_size]
    test = df[train_size:]
    return train, test

In [130]:
train_cat, test_cat = train_test_split(df['marka_pop'], train_size)
train_text, test_text = train_test_split(df['marka'], train_size)

In [131]:
y_test

array([6, 6, 6, ..., 3, 3, 2])

In [180]:
max_words = 200
tokenize = keras.preprocessing.text.Tokenizer(num_words = max_words, char_level = False)

In [181]:
tokenize.fit_on_texts(train_text)

x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)

In [182]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.fit_transform(train_cat)
y_test = encoder.fit_transform(test_cat)

In [183]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [184]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (10031, 200)
x_test shape: (2508, 200)
y_train shape: (10031, 54)
y_test shape: (2508, 54)


In [189]:
batch_size = 32
epochs = 2
drop_ratio = 0.5

In [190]:
model = models.Sequential()
model.add(layers.Dense(512, input_shape = (max_words,)))
model.add(layers.Activation('relu'))

model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [191]:
history = model.fit(x_train, y_train,
                   batch_size = batch_size,
                    epochs = 2,
                    verbose = 1,
                    validation_split = 0.8
                   )

Epoch 1/2
63/63 [==============================] - 1s 8ms/step - loss: 2.3785 - accuracy: 0.9053 - val_loss: 4.1381 - val_accuracy: 0.0287
Epoch 2/2
63/63 [==============================] - 0s 6ms/step - loss: 0.1371 - accuracy: 0.9880 - val_loss: 4.4762 - val_accuracy: 0.0285


In [192]:
def run_experiment(batch_size, epochs, drop_ratio):
  print('batch size: {}, epochs: {}, drop_ratio: {}'.format(
      batch_size, epochs, drop_ratio))
  model = models.Sequential()
  model.add(layers.Dense(512, input_shape=(max_words,)))
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(drop_ratio))
  model.add(layers.Dense(num_classes))
  model.add(layers.Activation('softmax'))

  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_split=0.1)
  score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=0)
  print('\tTest loss:', score[0])
  print('\tTest accuracy:', score[1])

In [195]:
batch_size = 50
epochs = 5
drop_ratio = 0.5
run_experiment(batch_size, epochs, drop_ratio)

batch size: 50, epochs: 5, drop_ratio: 0.5
	Test loss: 3.8560774326324463
	Test accuracy: 0.0


In [198]:
text_labels = encoder.classes_ 

for i in range(15):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_text.iloc[i][:50], "...")
    print('Actual label:' + test_cat.iloc[i])
    print("Predicted label: " + predicted_label + "\n")  


SATURN ...
Actual label:OPEL
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART

SEAT ...
Actual label:SEAT
Predicted label: SMART



In [200]:
prediction

array([[0.02759067, 0.05412173, 0.0119994 , 0.0263502 , 0.18642126,
        0.03889773, 0.20183057, 0.00941428, 0.00982748, 0.0088524 ,
        0.00932147, 0.00955747, 0.00867669, 0.00911953, 0.00978873,
        0.00861238, 0.00990461, 0.00979252, 0.01019414, 0.00897636,
        0.00940869, 0.01000494, 0.00905277, 0.00874915, 0.00938889,
        0.00820053, 0.00953651, 0.00956135, 0.01027953, 0.00983542,
        0.00960448, 0.00867443, 0.00947839, 0.01039154, 0.01036473,
        0.01086401, 0.01004962, 0.00981695, 0.01030339, 0.01002719,
        0.00864511, 0.00995019, 0.01025532, 0.01032149, 0.00881856,
        0.00923786, 0.00852811, 0.00942334, 0.00971805, 0.01419165,
        0.00943137, 0.01005135, 0.00881563, 0.00976979]], dtype=float32)